# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [9]:
import pandas as pd
import requests
import json

In [10]:
# Setting up the config for usage.
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    username = config['username']
    print("Config Loaded Successfully.")
    key_list = [key1, key2, key3]
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    username = input('Enter your username: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
#     with open('config.json.sample','r') as f:
#         config = json.load(f)
    config = {'user_agent':None,'key1':None, 'key2':None, 'key3':None, 'username':None}
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    config['username'] = username
    
    with open('config.json','w+') as f:
        json.dump(config, f)
    
    key_list = [key1, key2, key3]

    print()
    print("Your configuration has been saved to config.json!")
    
# Fetches variables: user_agent, username, key1, key2, key3, key_list

Config Loaded Successfully.


# Fetching Data from Last.fm

In [11]:
# Generic Function for sending requests

def send_request(user=username, page=1, limit=200, key=key1, method='user.getrecenttracks'):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':user,
    'page': page,
    'api_key': key,
    'method': method,
    'format': 'json'
    }
    
    api_reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return api_reply

In [12]:
# Fetch a page and clean data before return.
# Also handles exceptions.

def get_page(num=1, username=username, key=key1, method='user.getrecenttracks'):
    
    try:
        response = send_request(page=num, key=key, method=method)
        
        if response.status_code != 200:
            raise Exception(api_reply.text)
        
        page = (response.json())['recenttracks']['track']
        
        if num == 1: 
            global totalPages
            totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
        
        if (num!=1) & ('@attr' in page[0].keys()): 
            if (page[0]['@attr']=={'nowplaying': 'true'}):
                del page[0]
        
        return page

    except:
        raise Exception("request status {}".format(response.status_code))

# Get the total number of pages for 200 scrobbles per page.

def get_totalPages(method='user.getrecenttracks'):
    response = send_request(method = method)
    totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
    return totalPages

totalPages = get_totalPages()

In [13]:
# Picks a key from given list of keys
def key_picker(num, ls):
    index_num = num % len(ls)
    return ls[index_num]

In [14]:
# Fetches all the pages and merges them into a list "pages"
# Uses a fancy progress bar to show progress.
# Large accounts may time time to fetch!


from IPython.core.display import clear_output

pages = []

for n in range(1, (totalPages)+1):
    
    print('fetched page {}/{}'.format(n, totalPages))
    
    spaces = (((totalPages-n)/totalPages)*50)
    bar = ((n/totalPages)*50)
    string = '[{}{}] {}% Completed'.format('~'*int(bar),' '*int(spaces), int(bar)*2)
    print(string)
    
    clear_output(wait=True)
    
    ky = key_picker(num = n, ls = key_list)
    fetched_page = get_page(num=n, key=ky)
    
    pages = [*pages, *fetched_page]
print()
print("Fetched {} pages, {} scrobbles".format(totalPages, len(pages)))


Fetched 63 pages, 12574 scrobbles


# Cleaning up Fetched Data with #Pandas

In [15]:
df = pd.DataFrame(pages)
df.head()

,artist,streamable,image,mbid,album,name,url,date
0,"{'mbid': '', '#text': 'Yutaka Yamada'}",0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': '「ヴィンランド・サガ」オリジナル・サウンドトラ...",目覚め,https://www.last.fm/music/Yutaka+Yamada/_/%E7%...,"{'uts': '1643313381', '#text': '27 Jan 2022, 1..."
1,{'mbid': '60d2ea34-1912-425f-bf9c-fc544e4448cd...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",cba68b2c-0248-4d35-ada5-32743987bb96,{'mbid': '6ea5a900-67b4-489a-9130-754cdbfd805f...,Symphonicsuite (Aot) Part2-2nd: Shingekinokyojin,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,"{'uts': '1643313150', '#text': '27 Jan 2022, 1..."
2,{'mbid': '60d2ea34-1912-425f-bf9c-fc544e4448cd...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",3095d0ac-9e56-4b4b-b379-2ac960617040,{'mbid': '6ea5a900-67b4-489a-9130-754cdbfd805f...,Symphonicsuite (Aot) Part2-3rd: Before Lights Out,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,"{'uts': '1643312973', '#text': '27 Jan 2022, 1..."
3,{'mbid': 'f9114439-1662-4415-b761-05a4170c9579...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",16a09d38-8d76-4813-b341-7be8eae41d90,{'mbid': '00d3ce4a-fd63-49c1-9fc1-af98ea61a4f2...,LAY YOUR HANDS ON ME (2017),https://www.last.fm/music/BOOM+BOOM+SATELLITES...,"{'uts': '1643312577', '#text': '27 Jan 2022, 1..."
4,{'mbid': 'a0ecfe6b-1035-4d0a-b8c8-c75872139151...,0,"[{'size': 'small', '#text': 'https://lastfm.fr...",,"{'mbid': '', '#text': 'Enigmatic Feeling'}",Enigmatic Feeling,https://www.last.fm/music/Ling+Tosite+Sigure/_...,"{'uts': '1643305136', '#text': '27 Jan 2022, 1..."


In [16]:
df.drop(['image','streamable'], inplace=True, axis=1)
if '@attr' in df.columns:
  df.drop('@attr', inplace=True, axis = 1)
df.head()

,artist,mbid,album,name,url,date
0,"{'mbid': '', '#text': 'Yutaka Yamada'}",,"{'mbid': '', '#text': '「ヴィンランド・サガ」オリジナル・サウンドトラ...",目覚め,https://www.last.fm/music/Yutaka+Yamada/_/%E7%...,"{'uts': '1643313381', '#text': '27 Jan 2022, 1..."
1,{'mbid': '60d2ea34-1912-425f-bf9c-fc544e4448cd...,cba68b2c-0248-4d35-ada5-32743987bb96,{'mbid': '6ea5a900-67b4-489a-9130-754cdbfd805f...,Symphonicsuite (Aot) Part2-2nd: Shingekinokyojin,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,"{'uts': '1643313150', '#text': '27 Jan 2022, 1..."
2,{'mbid': '60d2ea34-1912-425f-bf9c-fc544e4448cd...,3095d0ac-9e56-4b4b-b379-2ac960617040,{'mbid': '6ea5a900-67b4-489a-9130-754cdbfd805f...,Symphonicsuite (Aot) Part2-3rd: Before Lights Out,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,"{'uts': '1643312973', '#text': '27 Jan 2022, 1..."
3,{'mbid': 'f9114439-1662-4415-b761-05a4170c9579...,16a09d38-8d76-4813-b341-7be8eae41d90,{'mbid': '00d3ce4a-fd63-49c1-9fc1-af98ea61a4f2...,LAY YOUR HANDS ON ME (2017),https://www.last.fm/music/BOOM+BOOM+SATELLITES...,"{'uts': '1643312577', '#text': '27 Jan 2022, 1..."
4,{'mbid': 'a0ecfe6b-1035-4d0a-b8c8-c75872139151...,,"{'mbid': '', '#text': 'Enigmatic Feeling'}",Enigmatic Feeling,https://www.last.fm/music/Ling+Tosite+Sigure/_...,"{'uts': '1643305136', '#text': '27 Jan 2022, 1..."


In [17]:
#Split the artist column, and rename the df. Drop the orginal column
artist_split = df['artist'].apply(pd.Series).rename(columns = {'mbid':'artist_mbid','#text':'artist'})
df.drop(columns = ['artist'], inplace=True)

#Split the album column, and rename the df. Drop the original column
album_split = df['album'].apply(pd.Series).rename(columns = {'mbid':'album_mbid','#text':'album'})
df.drop(columns = ['album'], inplace=True)

#Split the date column, and rename the df. Drop the original column
#Also check if residual '0' column is made. (If the top song is being streamed, it wont include the timing.)
date_split = df['date'].apply(pd.Series).rename(columns = {'uts':'uts','#text':'date'})
df.drop(columns = ['date'], inplace=True)
if 0 in date_split.columns:
    date_split.drop(0, axis=1, inplace=True)

#Merging the new columns into main df
df = pd.concat([df, artist_split, album_split, date_split],axis=1)
df.head()

,mbid,name,url,artist_mbid,artist,album_mbid,album,uts,date
0,,目覚め,https://www.last.fm/music/Yutaka+Yamada/_/%E7%...,,Yutaka Yamada,,「ヴィンランド・サガ」オリジナル・サウンドトラック,1643313381,"27 Jan 2022, 19:56"
1,cba68b2c-0248-4d35-ada5-32743987bb96,Symphonicsuite (Aot) Part2-2nd: Shingekinokyojin,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,60d2ea34-1912-425f-bf9c-fc544e4448cd,Hiroyuki Sawano,6ea5a900-67b4-489a-9130-754cdbfd805f,"""Attack on Titan"" Season 3 Original Soundtrack",1643313150,"27 Jan 2022, 19:52"
2,3095d0ac-9e56-4b4b-b379-2ac960617040,Symphonicsuite (Aot) Part2-3rd: Before Lights Out,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...,60d2ea34-1912-425f-bf9c-fc544e4448cd,Hiroyuki Sawano,6ea5a900-67b4-489a-9130-754cdbfd805f,"""Attack on Titan"" Season 3 Original Soundtrack",1643312973,"27 Jan 2022, 19:49"
3,16a09d38-8d76-4813-b341-7be8eae41d90,LAY YOUR HANDS ON ME (2017),https://www.last.fm/music/BOOM+BOOM+SATELLITES...,f9114439-1662-4415-b761-05a4170c9579,BOOM BOOM SATELLITES,00d3ce4a-fd63-49c1-9fc1-af98ea61a4f2,19972016 -20082016-,1643312577,"27 Jan 2022, 19:42"
4,,Enigmatic Feeling,https://www.last.fm/music/Ling+Tosite+Sigure/_...,a0ecfe6b-1035-4d0a-b8c8-c75872139151,Ling Tosite Sigure,,Enigmatic Feeling,1643305136,"27 Jan 2022, 17:38"


In [18]:
#Reordering columns
col_reorder = ['artist','name','album','date','artist_mbid','mbid','album_mbid','uts','url']
df = df.reindex(col_reorder, axis=1)
df.head()

,artist,name,album,date,artist_mbid,mbid,album_mbid,uts,url
0,Yutaka Yamada,目覚め,「ヴィンランド・サガ」オリジナル・サウンドトラック,"27 Jan 2022, 19:56",,,,1643313381,https://www.last.fm/music/Yutaka+Yamada/_/%E7%...
1,Hiroyuki Sawano,Symphonicsuite (Aot) Part2-2nd: Shingekinokyojin,"""Attack on Titan"" Season 3 Original Soundtrack","27 Jan 2022, 19:52",60d2ea34-1912-425f-bf9c-fc544e4448cd,cba68b2c-0248-4d35-ada5-32743987bb96,6ea5a900-67b4-489a-9130-754cdbfd805f,1643313150,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...
2,Hiroyuki Sawano,Symphonicsuite (Aot) Part2-3rd: Before Lights Out,"""Attack on Titan"" Season 3 Original Soundtrack","27 Jan 2022, 19:49",60d2ea34-1912-425f-bf9c-fc544e4448cd,3095d0ac-9e56-4b4b-b379-2ac960617040,6ea5a900-67b4-489a-9130-754cdbfd805f,1643312973,https://www.last.fm/music/Hiroyuki+Sawano/_/Sy...
3,BOOM BOOM SATELLITES,LAY YOUR HANDS ON ME (2017),19972016 -20082016-,"27 Jan 2022, 19:42",f9114439-1662-4415-b761-05a4170c9579,16a09d38-8d76-4813-b341-7be8eae41d90,00d3ce4a-fd63-49c1-9fc1-af98ea61a4f2,1643312577,https://www.last.fm/music/BOOM+BOOM+SATELLITES...
4,Ling Tosite Sigure,Enigmatic Feeling,Enigmatic Feeling,"27 Jan 2022, 17:38",a0ecfe6b-1035-4d0a-b8c8-c75872139151,,,1643305136,https://www.last.fm/music/Ling+Tosite+Sigure/_...


# Exporting into CSV

In [ ]:
df.to_csv('export.csv', encoding='utf-8')

# Data Analysis & Visualization

In [ ]:
# In Progress